# MLP Training Module

In [1]:
import os
os.chdir('genre_classification_289a/src')

In [2]:
import torch
import torch.nn as nn
import os
from model import STN, MLP
import torch.optim as optim

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
from features import get_data_loaders, FramedFeatureDataset, FeatureDataset, DatasetSettings

### MLP Training

In [5]:
#STN targets
agfs = [] #'subgenre', 'mfcc'
genre = True #False if not genre STN

target = None

if (genre and len(agfs) > 0) or (len(agfs) > 0):
    raise 'Error: can only train one STN target at a time'
elif genre:
    target = 'genre'
elif len(agfs) == 1:
    target = agfs[0]
else:
    raise 'Error: could not parse training target'

print('Training STN target: {}'.format(target))
    
#dataset
dataset_name = 'fma_small'

Training STN target: genre


In [6]:
settings = DatasetSettings(dataset_name, 'fma_metadata')
dataset = FramedFeatureDataset(settings,  agfs=agfs, genre=genre)
print("Num genres: ", settings.num_genres)
print(settings.genre_counts)

Num genres:  8
Pop              1000
International    1000
Instrumental     1000
Folk             1000
Rock              999
Experimental      999
Electronic        999
Hip-Hop           997
Name: genre_top, dtype: int64


In [7]:
def get_stn_path(dataset, target):
    return '../models/DCNN_{}_{}'.format(dataset, target)

# load STNs
stn1 = torch.load(get_stn_path('fma_small', 'genre'))

# setup MLP on GPU
mlp_input_size = 256
mlp_output_size = settings.num_genres
mlp = MLP(mlp_input_size, mlp_output_size)
mlp.to(device)
mlp = nn.DataParallel(mlp)

## Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
epochs = 5
batch_size = 64
valid_split = 0.2

trainloader, validloader = get_data_loaders(dataset, batch_size, valid_split)

In [8]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

def validate(mlp, label_name):
    
    with torch.no_grad():
        stn1.eval()
        mlp.eval()
                
        all_pred = []
        all_true = []
        
        for i, data in enumerate(validloader, 0):
            inputs, labels = data[0].to(device), data[1][label_name].to(device)
            
            out_intermediate = stn1.module.forward_intermediate(inputs, 7)
            input_mlp = out_intermediate
            
            out = mlp(input_mlp)
            loss = F.cross_entropy(out, labels)
            
            all_pred.append(out.argmax(dim=1))
            all_true.append(labels)
            
        all_pred = torch.cat(all_pred)
        all_true = torch.cat(all_true)
        
        curr_f1 = f1_score(all_true.cpu(), all_pred.cpu(), average='micro')
        
        print('Validation f1 total/micro score: {}'.format(curr_f1))

In [9]:
#Train it
stn1.eval()

for epoch in range(epochs):  # loop over the dataset multiple times
    
    print('Starting epoch', epoch + 1)
    
    validate(mlp, 'genre')
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1]['genre'].to(device)  #data[1]['{argument for agf being trained}']

        input_mlp = None
        with torch.no_grad():
            out_intermediate = stn1.module.forward_intermediate(inputs, 7)
            #input_mlp = out_intermediate.to(device)
            input_mlp = out_intermediate
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mlp(input_mlp)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 30 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
            
    validate(mlp, 'genre')
    mlp.train()

print('Finished Training')

Starting epoch 1
Validation f1 total/micro score: 0.15074613528728442
[1,    50] loss: 1.002
[1,   100] loss: 0.336
[1,   150] loss: 0.234
[1,   200] loss: 0.225
[1,   250] loss: 0.196
[1,   300] loss: 0.191
[1,   350] loss: 0.196
[1,   400] loss: 0.181
[1,   450] loss: 0.176
[1,   500] loss: 0.191
[1,   550] loss: 0.203
[1,   600] loss: 0.216
[1,   650] loss: 0.178
[1,   700] loss: 0.167
Validation f1 total/micro score: 0.9666696452506478
Starting epoch 2
Validation f1 total/micro score: 0.9666696452506478
[2,    50] loss: 0.137
[2,   100] loss: 0.142
[2,   150] loss: 0.127
[2,   200] loss: 0.122
[2,   250] loss: 0.130


KeyboardInterrupt: 

### Save Model

In [ ]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
torch.save(stn, model_file)

## Load & Eval Model

In [12]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
model = torch.load(model_file)
model

DataParallel(
  (module): STN(
    (layer1): Sequential(
      (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ELU(alpha=1.0)
      (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    )
    (layer2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
      (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.1, inplace=False)
    )
    (layer3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ELU(alpha=1.0)
      (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    )
    (layer4): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, mome

In [13]:
validate(model, target)

Validation f1 score: 0.9662228576534715


## Previous STN Val Accuracies
FMA_SMALL
* genre: 89.X% (can't reload model to validate b/c didn't set a random seed during training and the random split of training / validation data is different so we're mixing training data into the validation set and getting  unrealistically high validation accuracies)

FMA_MED
* genre: 89.3%
